In [ ]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib inline
import sys 
sys.path.append("../..")

import pykep as pk
import paseos
from paseos import ActorBuilder, SpacecraftActor
from paseos.utils.load_default_cfg import load_default_cfg
import asyncio
paseos.set_log_level("INFO")

In [ ]:
from get_constellation import get_constellation
altitude = 400 * 1000
inclination = 40.0
nPlanes = 2
nSats = 1
t0 = pk.epoch(0)

planet_list,sats_pos_and_v = get_constellation(altitude,inclination,nSats,nPlanes,t0)

In [ ]:
fig = plt.figure(figsize=(6,6),dpi=100)
ax = plt.axes(projection='3d')
for i in range (nPlanes*nSats):
    pk.orbit_plots.plot_planet(planet_list[i],axes=ax,s=64)

In [ ]:
paseos_instances = []
earth = pk.planet.jpl_lp("earth")
for idx,sat_pos_v in enumerate(sats_pos_and_v):
    pos,v = sat_pos_v
    sat_actor = ActorBuilder.get_actor_scaffold(name="Sat_"+str(idx),
                                                actor_type=SpacecraftActor,
                                                epoch=t0)
    ActorBuilder.set_orbit(actor=sat_actor,position=pos,velocity=v,epoch=t0,central_body=earth)
    ActorBuilder.add_comm_device(actor=sat_actor,device_name="Link1",bandwidth_in_kbps=1000)
    ActorBuilder.set_power_devices(actor=sat_actor,battery_level_in_Ws=5000,
                                   max_battery_level_in_Ws=100000,charging_rate_in_W=50)
    ActorBuilder.set_thermal_model(
            actor=sat_actor,
            actor_mass=50.0,
            actor_initial_temperature_in_K=273.15,
            actor_sun_absorptance=1.0,
            actor_infrared_absorptance=1.0,
            actor_sun_facing_area=1.0,
            actor_central_body_facing_area=1.0,
            actor_emissive_area=1.0,
            actor_thermal_capacity=1000
    )
    
    instance = paseos.init_sim(local_actor=sat_actor)
    paseos_instances.append(instance)

In [ ]:
def get_consumption_and_activity(instance):
    """Determine power consumption of the instance"""
    if operational_constraint(instance):
        return 30.0,"Processing"
    else: 
        return 3.0, "Charging"

def operational_constraint(instance):
    if (instance.local_actor.state_of_charge > 0.5 
        and instance.local_actor.temperature_in_K < 350):
        return True
    else:
        return False

In [ ]:
simulation_time = 1.0 * 3600 # one day in seconds
t = 0
timestep = 60 # 1min
while t < simulation_time: 
    # For each satellite
    N_charging = 0
    N_processing = 0
    for instance in paseos_instances:
        # Determine whether satellite is performing activity
        power_consumption, activity = get_consumption_and_activity(instance)
        if activity == "Processing":
            N_processing += 1
            eval_constraint = lambda: operational_constraint(instance)
        else:
            N_charging += 1
            eval_constraint = None
        instance.advance_time(
            time_to_advance=timestep,
            current_power_consumption_in_W=power_consumption,
            constraint_function=eval_constraint
        )
    print(f"Time:{t} - # of Processing = {N_processing}, # of Charging = {N_charging}")
    t += timestep

In [ ]:
instance._operations_monitor._log.state_of_charge